In [323]:
from __future__ import print_function
import geopandas as gpd
import pandas as pd
import numpy as np
import urllib
import requests
import os
import io
import json
import pylab as pl
import shapely
import shapely.wkt
from shapely.geometry import Point
from fiona.crs import from_epsg
import sys
import choropleth as cp
from censusapi import myapi
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Downloading shapefile for Census Tracts 

In [272]:
url = 'https://data.cityofnewyork.us/api/geospatial/fxpq-c8ku?method=export&format=Shapefile'
urllib.request.urlretrieve(url, "file.gz")
# unpacking into $PUIDATA
!unzip file.gz -d $PUIDATA

Archive:  file.gz
  inflating: /nfshome/cb4221/PUIdata/geo_export_561dfd54-713b-4db1-b9f1-1d80b6628047.dbf  
  inflating: /nfshome/cb4221/PUIdata/geo_export_561dfd54-713b-4db1-b9f1-1d80b6628047.shp  
  inflating: /nfshome/cb4221/PUIdata/geo_export_561dfd54-713b-4db1-b9f1-1d80b6628047.shx  
  inflating: /nfshome/cb4221/PUIdata/geo_export_561dfd54-713b-4db1-b9f1-1d80b6628047.prj  


In [273]:
tracts = gpd.GeoDataFrame.from_file("%s/geo_export_bfc6efa1-df03-48d2-a9ea-3b3db53d4be8.shp"%os.getenv("PUIDATA"))

In [274]:
tracts.head(2)

,boro_code,boro_ct201,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,geometry
0,5,5000900,Staten Island,I,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,POLYGON ((-74.07920577013245 40.64343078374567...
1,1,1009800,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.199811,POLYGON ((-73.96432543478758 40.75638153099091...


## Download police precincts shapefile

In [275]:
url = 'https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=Shapefile'
urllib.request.urlretrieve(url, "file.gz")
!unzip file.gz -d $PUIDATA

Archive:  file.gz
  inflating: /nfshome/cb4221/PUIdata/geo_export_dee92f77-245d-4615-b14f-28dec73bee7b.dbf  
  inflating: /nfshome/cb4221/PUIdata/geo_export_dee92f77-245d-4615-b14f-28dec73bee7b.shp  
  inflating: /nfshome/cb4221/PUIdata/geo_export_dee92f77-245d-4615-b14f-28dec73bee7b.shx  
  inflating: /nfshome/cb4221/PUIdata/geo_export_dee92f77-245d-4615-b14f-28dec73bee7b.prj  


In [276]:
precincts = gpd.GeoDataFrame.from_file("%s/geo_export_5ae57718-dcea-4aac-afac-286bc14b8682.shp"%os.getenv('PUIDATA'))

In [277]:
precincts.head()

,precinct,shape_area,shape_leng,geometry
0,1.0,4.730176e+07,80586.154615,(POLYGON ((-74.0438776157395 40.69018767637665...
1,5.0,1.808880e+07,18676.124259,POLYGON ((-73.98863862848766 40.72293372026369...
2,6.0,2.209819e+07,26402.900669,POLYGON ((-73.99968392160721 40.73855224865976...
3,71.0,4.533179e+07,29978.094261,POLYGON ((-73.92854313809303 40.66457328584737...
4,72.0,1.046213e+08,87968.194520,POLYGON ((-73.99840899113158 40.67186872303234...


## Reading in vacancy data, downloaded directly from FactFinder
To download:
- go to https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml
- select advanced search
- search for table B25004
- select 2017 5-year estimates
- in Add/Remove Geographies, select Census Tract, New York state
- Select all census tracts within these counties: Bronx, Kings, New York, Queens, and Richmond

FIPS codes for 5 boroughs
- 005 - Bronx
- 047 - Kings (Brooklyn)
- 061 - New York (Manhattan)
- 081 - Queens
- 085 - Richmond (Staten Island) 

In [278]:
vacancy = pd.read_csv('ACS_17_5YR_B25004.csv')
vacancy.drop(0, axis=0, inplace=True) # dropping information first row

In [336]:
# # API call, but just provides one column

# url = "https://api.census.gov/data/2017/acs/acs5?get=" + 'B25004_001E' + \
# ",NAME&for=tract:*&in=state:36&in=county:005&key=" + myapi
# resp = requests.request('GET', url).content
# total = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))

In [337]:
# total.head()

## Creating common column
Vacancy and Tracts need to be joined on common column, but don't have one. My solution is to create a boro_code in vacancy and slice part of the GEO id to create two columns to merge on, since I cannot create one unique column

- Manhattan = 1
- Bronx = 2
- Brooklyn = 3
- Queens = 4
- Staten Island = 5

## Add borough code to vacancy df

In [279]:
# splitting 'Geo display label' into 3 parts
vacancy[['tract', 'county', 'state']] = pd.DataFrame([ x.split(',') for x in vacancy['GEO.display-label'].tolist()])

In [280]:
vacancy.county.unique()

array([' Bronx County', ' Kings County', ' New York County',
       ' Queens County', ' Richmond County', nan], dtype=object)

In [281]:
vacancy.county = vacancy.county.astype(str)

In [282]:
mymap = {' New York County': 1, ' Bronx County': 2, ' Kings County':3, ' Queens County':4, 
          ' Richmond County':5}

In [283]:
# replaces 'New York County' etc. with corresponding mapped number 
vacancyR = vacancy.applymap(lambda s: mymap.get(s) if s in mymap else s)
vacancyR.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD01_VD02,HD01_VD03,HD01_VD04,HD01_VD05,HD01_VD06,HD01_VD07,HD01_VD08,tract,county,state
1,1400000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",0,0,0,0,0,0,0,0,Census Tract 2,2,New York
2,1400000US36005000200,36005000200,"Census Tract 2, Bronx County, New York",144,0,0,16,0,0,0,128,Census Tract 4,2,New York
3,1400000US36005000400,36005000400,"Census Tract 4, Bronx County, New York",183,72,0,31,35,0,0,45,Census Tract 16,2,New York
4,1400000US36005001600,36005001600,"Census Tract 16, Bronx County, New York",83,22,41,0,0,0,0,20,Census Tract 19,2,New York
5,1400000US36005001900,36005001900,"Census Tract 19, Bronx County, New York",48,10,0,0,0,10,0,28,Census Tract 20,2,New York


# taking slice of GEO.id2 to match 'ct2010' column in tracts 

In [284]:
vacancyR['ct2010'] = vacancyR['GEO.id2'].str[5:]

In [285]:
vacancyR.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD01_VD02,HD01_VD03,HD01_VD04,HD01_VD05,HD01_VD06,HD01_VD07,HD01_VD08,tract,county,state,ct2010
1,1400000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",0,0,0,0,0,0,0,0,Census Tract 2,2,New York,000100
2,1400000US36005000200,36005000200,"Census Tract 2, Bronx County, New York",144,0,0,16,0,0,0,128,Census Tract 4,2,New York,000200
3,1400000US36005000400,36005000400,"Census Tract 4, Bronx County, New York",183,72,0,31,35,0,0,45,Census Tract 16,2,New York,000400
4,1400000US36005001600,36005001600,"Census Tract 16, Bronx County, New York",83,22,41,0,0,0,0,20,Census Tract 19,2,New York,001600
5,1400000US36005001900,36005001900,"Census Tract 19, Bronx County, New York",48,10,0,0,0,10,0,28,Census Tract 20,2,New York,001900


In [286]:
vacancyR.shape

(2167, 15)

In [287]:
tracts.shape

(2166, 12)

In [288]:
vacancyR.dropna(axis=0, how='any', inplace=True) # there was one NaN

# making sure I have all the same data types

In [289]:
vacancyR.ct2010 = vacancyR.ct2010.astype(int)

In [290]:
vacancyR.county = vacancyR.county.astype(int)

In [291]:
tracts.boro_code = tracts.boro_code.astype(int)

In [292]:
tracts.ct2010 = tracts.ct2010.astype(int)

## Merging on two common columns, ct2010 and boro/county code
- also column renaming

In [293]:
vacancyTracts = pd.merge(vacancyR, tracts,  how='inner', 
                         left_on=['ct2010', 'county'], right_on = ['ct2010','boro_code'])

In [294]:
vacancyTracts.shape

(2163, 26)

In [295]:
vacancyTracts.columns

Index(['GEO.id', 'GEO.id2', 'GEO.display-label', 'HD01_VD01', 'HD01_VD02',
       'HD01_VD03', 'HD01_VD04', 'HD01_VD05', 'HD01_VD06', 'HD01_VD07',
       'HD01_VD08', 'tract', 'county', 'state', 'ct2010', 'boro_code',
       'boro_ct201', 'boro_name', 'cdeligibil', 'ctlabel', 'ntacode',
       'ntaname', 'puma', 'shape_area', 'shape_leng', 'geometry'],
      dtype='object')

In [296]:
vacancyTracts = vacancyTracts[['GEO.display-label', 'HD01_VD01', 'HD01_VD02',
       'HD01_VD03', 'HD01_VD04', 'HD01_VD05', 'HD01_VD06', 'HD01_VD07',
       'HD01_VD08', 'shape_area', 'shape_leng', 'geometry']]

In [297]:
vacancyTracts.rename(columns={'HD01_VD01':'total', 'HD01_VD02':'rentals', 'HD01_VD03':'rented_vacant', 
                        'HD01_VD04':'for_sale', 'HD01_VD05':'sold_vacanct', 'HD01_VD06':'seasonal',
                       'HD01_VD07':'migrant_wrks', 'HD01_VD08':'other'}, inplace=True)

In [298]:
vacancyTracts.head()

,GEO.display-label,total,rentals,rented_vacant,for_sale,sold_vacanct,seasonal,migrant_wrks,other,shape_area,shape_leng,geometry
0,"Census Tract 1, Bronx County, New York",0,0,0,0,0,0,0,0,1.816383e+07,18898.116621,POLYGON ((-73.87287195903875 40.78597502780474...
1,"Census Tract 2, Bronx County, New York",144,0,0,16,0,0,0,128,5.006558e+06,15610.702268,POLYGON ((-73.85651604096793 40.80524122047598...
2,"Census Tract 4, Bronx County, New York",183,72,0,31,35,0,0,45,8.561175e+06,24725.471690,POLYGON ((-73.84610660457847 40.81309998920543...
3,"Census Tract 16, Bronx County, New York",83,22,41,0,0,0,0,20,5.221330e+06,9671.306205,POLYGON ((-73.85513639815333 40.82243618931003...
4,"Census Tract 19, Bronx County, New York",48,10,0,0,0,10,0,28,1.796136e+07,29999.575405,(POLYGON ((-73.89680883223774 40.7958084451597...


In [299]:
type(precincts)

geopandas.geodataframe.GeoDataFrame

In [300]:
type(vacancyTracts)

pandas.core.frame.DataFrame

# Merging vacancy info at tract level with precincts 

In [301]:
# first turning into gdf
vacancyTractsG = gpd.GeoDataFrame(vacancyTracts, geometry='geometry')

In [305]:
vacancyTractsG.crs

In [306]:
vacancyTractsG.crs = from_epsg(4326)
vacancyTractsG.to_crs(epsg=2263, inplace=True)

In [308]:
precincts.crs = from_epsg(4326)
precincts.to_crs(epsg=2263, inplace=True)

In [317]:
# merging
vacancy_by_precinct2 = gpd.sjoin(precincts, vacancyTractsG, how="inner", op='contains')

In [321]:
vacancy_by_precinct2.head(20)

,precinct,shape_area_left,shape_leng_left,geometry,index_right,GEO.display-label,total,rentals,rented_vacant,for_sale,sold_vacanct,seasonal,migrant_wrks,other,shape_area_right,shape_leng_right
0,1.0,4.730176e+07,80586.154615,(POLYGON ((972081.7882080087 190733.4674071776...,1109,"Census Tract 13, New York County, New York",592,116,57,40,102,209,0,68,3.516583e+06,8906.314403
1,5.0,1.808880e+07,18676.124259,"POLYGON ((987399.20678711 202660.9920043435, 9...",1121,"Census Tract 25, New York County, New York",29,7,0,0,0,0,0,22,1.439388e+06,4763.702704
1,5.0,1.808880e+07,18676.124259,"POLYGON ((987399.20678711 202660.9920043435, 9...",1124,"Census Tract 27, New York County, New York",76,17,0,0,8,22,0,29,6.392992e+05,3685.315982
1,5.0,1.808880e+07,18676.124259,"POLYGON ((987399.20678711 202660.9920043435, 9...",1139,"Census Tract 41, New York County, New York",395,45,31,0,0,277,0,42,2.058620e+06,5913.871153
2,6.0,2.209819e+07,26402.900669,"POLYGON ((984337.591796876 208351.1055907656, ...",1150,"Census Tract 55.01, New York County, New York",307,48,49,0,0,163,0,47,1.338497e+06,4650.767118
2,6.0,2.209819e+07,26402.900669,"POLYGON ((984337.591796876 208351.1055907656, ...",1161,"Census Tract 65, New York County, New York",437,185,35,0,0,111,0,106,2.167562e+06,6351.231521
2,6.0,2.209819e+07,26402.900669,"POLYGON ((984337.591796876 208351.1055907656, ...",1163,"Census Tract 67, New York County, New York",486,164,60,0,124,0,0,138,1.812692e+06,5458.908916
2,6.0,2.209819e+07,26402.900669,"POLYGON ((984337.591796876 208351.1055907656, ...",1155,"Census Tract 59, New York County, New York",398,130,0,0,0,216,0,52,1.250316e+06,5280.102496
2,6.0,2.209819e+07,26402.900669,"POLYGON ((984337.591796876 208351.1055907656, ...",1169,"Census Tract 73, New York County, New York",851,118,127,124,0,341,0,141,1.778584e+06,5754.134334
3,71.0,4.533179e+07,29978.094261,"POLYGON ((1004074.144409176 181406.4476317891,...",973,"Census Tract 874.01, Kings County, New York",126,0,0,0,0,0,0,126,1.405623e+06,5380.177876
